### aim: 

### date: 

In [2]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [3]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [4]:
import numpy as np
from cmocean import cm
import cartopy as cp
import cartopy.crs as ccrs
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr

%matplotlib inline
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import cartopy.feature as cfeature
from importlib import reload
import matplotlib.path as mpath
import glob
import pickle
import pandas as pd
import seawater
import time
plt.rcParams.update({'font.size': 13})
font = {'family' : 'normal',
'weight' : 'normal',
'size'   : 13}
plt.rcParams['text.usetex'] = True
plt.rc('font', **font)

import sys
sys.path.append('/gpfs/home/mep22dku/scratch/SOZONE/UTILS')
import snippets as sp
reload(sp)
import gsw

In [12]:
diad = xr.open_dataset('/gpfs/data/greenocean/software/runs/TOM12_TJ_R4A1/ORCA2_1m_19710101_19711231_diad_T.nc')
gridT = xr.open_dataset('/gpfs/data/greenocean/software/runs/TOM12_TJ_R4A1/ORCA2_1m_19710101_19711231_grid_T.nc')



### missing fields:

    3D temperature (thetao)
    3D Salinity (so)
    Mixed layer depth (mld) should be 0.03 density criterion 
    Potential Density
    3D dissolved inorganic carbon (dissc)
    3D Alkalinity (Talk)
    CFC11
    CFC12
    SF6
    Surface Chlorophyll (chlos)
    
### what needs doing

- calculate potential density 
- calc(?) MLD
- redo formatting

## formatting instructions

Formatting: We require that the NetCDF Climate and Forecast (CF) conventions for metadata in a
netCDF file are used (https://cfconventions.org/Data/cf-conventions/cf- conventions-1.11/cfconventions.pdf). Please make sure that CF conventions are followed also for variables of derived
types (e.g., AMOC, SO_SSS).

    Example for file with minimum metadata requirements:
    dimensions:
    time = UNLIMITED ; // (120 currently)
    bnds = 2 ;
    depth = 45 ;
    lat = 180 ;
    lon = 360 ;
    variables:
    double time(time) ; ### Please provide a calendar and units
    time:bounds = "time_bnds" ; ### See here for more details
    time:units = "days since 1959-01-01 00:00:00" ;
    time:calendar = "standard" ;
    time:axis = "T" ;
    time:long_name = "time" ;
    time:standard_name = "time" ;
    double time_bnds(time, bnds) ; ### For time_bnds, see here
    double lat(lat) ; ### Please see here for “lat” details
    lat:bounds = "lat_bnds" ; ### Please use “lat” as name, not “Y”, “ytocean” etc.
    lat:units = "degrees_north" ;
    lat:axis = "Y" ;
    lat:long_name = "Latitude" ;
    lat:standard_name = "latitude" ;
    double lat_bnds(lat, bnds) ; ### For lat_bnds and lon_bnds, see here
    double lon(lon) ; ### Please see here for more details
    lon:bounds = "lon_bnds" ; ### use “lon” as name, not “X”, “xtocean” etc.
    lon:units = "degrees_east" ;
    lon:axis = "X" ;
    lon:long_name = "Longitude" ;
    lon:standard_name = "longitude" ;
    double lon_bnds(lon, bnds) ;
    double depth(depth) ; ### For depth, see here
    depth:bounds = "depth_bnds" ;
    depth:units = "m" ;
    depth:axis = "Z" ;
    depth:positive = "down" ;
    depth:long_name = "ocean depth coordinate" ;
    depth:standard_name = "depth" ;
    double depth_bnds(depth, bnds) ;
    float thetao(time, depth, lat, lon) ;
    thetao:standard_name = "sea_water_potential_temperature" ;
    thetao:long_name = "Sea Water Potential Temperature" ;
    thetao:units = "degC" ; ### Please provide “units”
    thetao:cell_methods = "area: mean where sea time: mean" ;
    thetao:cell_measures = "area: areacello volume: volcello" ;
    thetao:_FillValue = 1.e+20f ; ### Please provide a FillValue or missing value
    // global attributes: “Please provide global attributes accordingly”

## code to open a test file

In [6]:
def open_file(tdir, tvar, abcd):

    tfil = glob.glob(f'{tdir}/{tvar}_PlankTOM_{abcd}_1_gr_*.nc')
    w = xr.open_dataset(tfil[0])
    
    return w

tdir = '/gpfs/data/greenocean/GCB/GCB2024/submissionData2024/RIV12_R4A-D1/'
tvar = 'thetao'
abcd = 'A'

w = open_file(tdir, tvar, abcd)



## reformatting

In [15]:
gridT

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, deptht: 31, time_counter: 12, x: 182, y: 149)
Coordinates:
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
  * deptht                (deptht) float32 5.0 15.0 25.0 ... 4.75e+03 5.25e+03
    time_centered         (time_counter) object ...
  * time_counter          (time_counter) object 1971-01-16 12:00:00 ... 1971-...
Dimensions without coordinates: axis_nbounds, x, y
Data variables: (12/20)
    deptht_bounds         (deptht, axis_nbounds) float32 ...
    e3t                   (time_counter, deptht, y, x) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) object ...
    time_counter_bounds   (time_counter, axis_nbounds) object ...
    votemper              (time_counter, deptht, y, x) float32 ...
    vosaline              (time_counter, deptht, y, x) float32 ...
    ...                    ...
    tohfls                (time_counter, y, x) float32 ...
    sosflxdo              (time_counter, y, x) float32 ...
    taum                  (time_counter, y, x) float32 ...
    sowindsp              (time_counter, y, x) float32 ...
    soprecip              (time_counter, y, x) float32 ...
    sbt                   (time_counter, y, x) float32 ...
Attributes:
    name:         ORCA2_1m_19710101_19711231
    description:  ocean T grid variables
    title:        ocean T grid variables
    Conventions:  CF-1.5
    production:   An IPSL model
    timeStamp:    2024-Jul-05 21:47:23 BST

In [16]:
gridT.votemper

<xarray.DataArray 'votemper' (time_counter: 12, deptht: 31, y: 149, x: 182)>
[10087896 values with dtype=float32]
Coordinates:
    nav_lat        (y, x) float32 -78.19 -78.19 -78.19 -78.19 ... 0.0 0.0 0.0
    nav_lon        (y, x) float32 78.0 80.0 82.0 84.0 86.0 ... 0.0 0.0 0.0 0.0
  * deptht         (deptht) float32 5.0 15.0 25.0 ... 4.25e+03 4.75e+03 5.25e+03
    time_centered  (time_counter) object 1971-01-16 12:00:00 ... 1971-12-16 1...
  * time_counter   (time_counter) object 1971-01-16 12:00:00 ... 1971-12-16 1...
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_potential_temperature
    long_name:           temperature
    units:               degC
    online_operation:    average
    interval_operation:  5760 s
    interval_write:      1 month
    cell_methods:        time: mean (interval: 5760 s)

In [36]:
def reformat_output(w, tvar, \
                    standard_name = 'sea_water_potential_temperature', units = 'degC'):
    
    today = datetime.today().strftime('%Y%m%d')
    print(today)

    ref_date = np.datetime64("1959-01-01T00:00:00")

    # Convert datetime64 to number of days since ref_date
    time_in_days = (w['TIME'].values - ref_date) / np.timedelta64(1, 'D')
    print(time_in_days[0:20])

    # Dimensions
    time = time_in_days
    depth = w.DEPTH.values  # example depth values
    lat = w.LATITUDE.values
    lon = w.LONGITUDE.values

    # Bounds
    time_bnds = np.column_stack([time, time + 1])  # simplistic 1-day bounds
    lat_bnds = np.column_stack([lat - 0.5, lat + 0.5])
    lon_bnds = np.column_stack([lon - 0.5, lon + 0.5])
    depth_bnds = np.column_stack([depth - 5, depth + 5])  # placeholder range

    # Data variable (dummy data)
    thetao_data = w.tvar.values

    # Create Dataset
    ds = xr.Dataset(
        {
            "thetao": (("time", "depth", "lat", "lon"), thetao_data),
            "time_bnds": (("time", "bnds"), time_bnds),
            "lat_bnds": (("lat", "bnds"), lat_bnds),
            "lon_bnds": (("lon", "bnds"), lon_bnds),
            "depth_bnds": (("depth", "bnds"), depth_bnds),
        },
        coords={
            "time": ("time", time, {
                "bounds": "time_bnds",
                "units": "days since 1959-01-01 00:00:00",
                "calendar": "standard",
                "axis": "T",
                "long_name": "time",
                "standard_name": "time"
            }),
            "lat": ("lat", lat, {
                "bounds": "lat_bnds",
                "units": "degrees_north",
                "axis": "Y",
                "long_name": "Latitude",
                "standard_name": "latitude"
            }),
            "lon": ("lon", lon, {
                "bounds": "lon_bnds",
                "units": "degrees_east",
                "axis": "X",
                "long_name": "Longitude",
                "standard_name": "longitude"
            }),
            "depth": ("depth", depth, {
                "bounds": "depth_bnds",
                "units": "m",
                "axis": "Z",
                "positive": "down",
                "long_name": "ocean depth coordinate",
                "standard_name": "depth"
            }),
        },
        attrs={
            # Add your global attributes here
            "title": f'{tvar} ,
            "summary": "Minimal metadata template with CF-compliant structure.",
            "Conventions": "CF-1.8",
            "institution": "GreenOcean group, University of East Anglia",
            "source": "Synthetic data",
            "history": "created on ",
            "references": "",
        }
    )

    # Add attributes to variable
    ds[tvar].attrs = {
        "standard_name": standard_name,
        "long_name": long_name, 
        "units": units,
        "cell_methods": "area: regrid using scipy.griddata",
        "cell_measures": "see AncillaryData_mask.nc",
        "_FillValue": 'NaN'
    }

    # Show summary
    print(ds)

[ 11.  42.  70. 101. 131. 162. 192. 223. 254. 284. 315. 345. 376. 407.
 436. 467. 497. 528. 558. 589.]


<xarray.DataArray 'LONGITUDE' (LONGITUDE: 360)>
array([  0.5,   1.5,   2.5, ..., 357.5, 358.5, 359.5], dtype=float32)
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5

In [17]:
w = xr.open_dataset('/gpfs/data/greenocean/GCB/GCB2023/submissionData2023/CAL12/Tier1/nativeGridDimensions.nc')

In [18]:
w

<xarray.Dataset>
Dimensions:          (GLOBE: 1, REGION: 4)
Coordinates:
  * REGION           (REGION) int64 0 1 2 3
  * GLOBE            (GLOBE) float64 0.0
Data variables:
    Area_tot_native  (REGION) float64 ...
    Vol_tot_native   (GLOBE) float64 ...
Attributes:
    made in:  /gpfs/data/greenocean/GCB/GCB2023/scripts/makeAncillary.ipynb
    desc:     yearly medusa files, saving only variables of interest

## calculate potential density - put in script 
CT = gsw_CT_from_pt(SA,pt)
rho = gsw_rho(SA,CT,p)
gsw_CT_from_pt


In [ ]:
ex = False
if ex: 
    import xarray as xr
    import glob
    import gsw
    from datetime import datetime

    today = datetime.today().strftime('%Y%m%d')


    def open_file(tdir, tvar, abcd):

        tfil = glob.glob(f'{tdir}/{tvar}_PlankTOM_{abcd}_1_gr_*.nc')
        w = xr.open_dataset(tfil[0])

        return w

    tdir = '/gpfs/data/greenocean/GCB/GCB2024/submissionData2024/RIV12_R4A-D1/'
    abcd = 'A'

    for i in range(0,780):
        print(i)
        tvar = 'thetao'
        thetao = open_file(tdir, tvar, abcd)
        tvar = 'so'
        so = open_file(tdir, tvar, abcd)

        thetao_val = thetao.thetao[i:i+1,:,:,:]
        so_val = so.so[i:i+1,:,:,:]

        rho = gsw.rho(so_val.values,thetao_val.values,0)

        # Check that 'thetao' exists
        if "thetao" not in thetao:
            raise KeyError("Variable 'thetao' not found in the dataset.")

        # Get shape and dtype of 'thetao'
        shape = thetao_val.shape
        dtype = thetao_val.dtype

        # Create a new variable 'rho' with the same shape and type as 'thetao'
        # Example: here we just fill with random numbers, you can change this to meaningful values
        rho_data = xr.DataArray(rho, 
                                coords=thetao_val.coords, 
                                dims=thetao_val.dims,
                                name="rho")

        # Create a copy of the dataset without 'thetao'
        ds_new = thetao.drop_vars("thetao")

        # Add 'rho' to the dataset
        ds_new["rho"] = rho_data

        # Save to a new NetCDF file
        ds_new.to_netcdf(f'{tdir}/rho_PlankTOM_{abcd}_1_gr_1959-2023_v{today}_chunk-{i}.nc')
        print(f'rho_PlankTOM_{abcd}_1_gr_1959-2023_v{today}_chunk-{i}.nc')
        thetao.close()
        so.close()

0


In [12]:
thetao

<xarray.Dataset>
Dimensions:    (DEPTH: 31, LATITUDE: 180, LONGITUDE: 360, TIME: 780)
Coordinates:
  * LONGITUDE  (LONGITUDE) float32 0.5 1.5 2.5 3.5 ... 356.5 357.5 358.5 359.5
  * LATITUDE   (LATITUDE) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * TIME       (TIME) datetime64[ns] 1959-01-12 1959-02-12 ... 2023-12-12
  * DEPTH      (DEPTH) float32 5.0 15.0 25.0 ... 4.25e+03 4.75e+03 5.25e+03
Data variables:
    thetao     (TIME, DEPTH, LATITUDE, LONGITUDE) float32 ...

https://www.teos-10.org/pubs/gsw/html/gsw_CT_from_pt.html

In [24]:
help(gsw.CT_from_pt)

Help on function CT_from_pt in module gsw._wrapped_ufuncs:

CT_from_pt(SA, pt)
    Calculates Conservative Temperature of seawater from potential
    temperature (whose reference sea pressure is zero dbar).
    
    Parameters
    ----------
    SA : array-like
        Absolute Salinity, g/kg
    pt : array-like
        Potential temperature referenced to a sea pressure, degrees C
    
    Returns
    -------
    CT : array-like, deg C
        Conservative Temperature (ITS-90)



In [25]:
w.TIME

ref_date = np.datetime64("1959-01-01T00:00:00")

# Convert datetime64 to number of days since ref_date
time_in_days = (ds['TIME'].values - ref_date) / np.timedelta64(1, 'D')


<xarray.DataArray 'TIME' (TIME: 780)>
array(['1959-01-12T00:00:00.000000000', '1959-02-12T00:00:00.000000000',
       '1959-03-12T00:00:00.000000000', ..., '2023-10-11T23:57:52.000000000',
       '2023-11-12T00:00:00.000000000', '2023-12-12T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * TIME     (TIME) datetime64[ns] 1959-01-12 1959-02-12 ... 2023-12-12
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_counter_bounds
    time_origin:    1950-JAN-01 00:00:00

In [3]:
w = xr.open_dataset('/gpfs/data/greenocean/software/runs/TOM12_TJ_R4A1/ORCA2_1m_19740101_19741231_diad_T.nc')

In [5]:
w

<xarray.Dataset>
Dimensions:               (axis_nbounds: 2, deptht: 31, time_counter: 12, x: 182, y: 149)
Coordinates:
    nav_lat               (y, x) float32 ...
    nav_lon               (y, x) float32 ...
  * deptht                (deptht) float32 5.0 15.0 25.0 ... 4.75e+03 5.25e+03
    time_centered         (time_counter) object ...
  * time_counter          (time_counter) object 1974-01-16 12:00:00 ... 1974-...
Dimensions without coordinates: axis_nbounds, x, y
Data variables: (12/40)
    deptht_bounds         (deptht, axis_nbounds) float32 0.0 10.0 ... 5.501e+03
    DOCTRP                (time_counter, deptht, y, x) float32 ...
    time_centered_bounds  (time_counter, axis_nbounds) object ...
    time_counter_bounds   (time_counter, axis_nbounds) object ...
    CARBTRP               (time_counter, deptht, y, x) float32 ...
    CAPITRP               (time_counter, deptht, y, x) float32 ...
    ...                    ...
    Cflx                  (time_counter, y, x) float32 ...
    PICflx                (time_counter, y, x) float32 ...
    Oflx                  (time_counter, y, x) float32 ...
    dpCO2                 (time_counter, y, x) float32 ...
    pCO2                  (time_counter, y, x) float32 ...
    PPINT                 (time_counter, y, x) float32 ...
Attributes:
    name:         ORCA2_1m_19740101_19741231
    description:  additional planktom diagnostics
    title:        additional planktom diagnostics
    Conventions:  CF-1.5
    production:   An IPSL model
    timeStamp:    2024-Jul-05 23:48:09 BST

In [21]:
w = xr.open_dataset('/gpfs/data/greenocean/GCB/GCB2023/submissionData2023/CAL12/Tier1/AncillaryData_mask.nc')

In [22]:
w

<xarray.Dataset>
Dimensions:        (DEPTH: 31, LATITUDE: 180, LONGITUDE: 360)
Coordinates:
  * LONGITUDE      (LONGITUDE) float32 0.5 1.5 2.5 3.5 ... 357.5 358.5 359.5
  * LATITUDE       (LATITUDE) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * DEPTH          (DEPTH) float32 5.0 15.0 25.0 ... 4.25e+03 4.75e+03 5.25e+03
Data variables:
    AREA           (LATITUDE, LONGITUDE) float32 ...
    VOLUME         (DEPTH, LATITUDE, LONGITUDE) float32 ...
    AREA_MASKED    (LATITUDE, LONGITUDE) float32 ...
    VOLUME_MASKED  (DEPTH, LATITUDE, LONGITUDE) float32 ...
    MASK_SFC       (LATITUDE, LONGITUDE) float32 ...
    MASK_VOL       (DEPTH, LATITUDE, LONGITUDE) float32 ...